# Quantum Kernel Methods for IRIS dataset classification with [TorchQuantum](https://github.com/mit-han-lab/torchquantum).
<p align="left">
<img src="https://github.com/mit-han-lab/torchquantum/blob/master/torchquantum_logo.jpg?raw=true" alt="torchquantum Logo" width="250">
</p>

Tutorial Author: Zirui Li, Hanrui Wang


###Outline
1. Introduction to Quantum Kernel Methods.
2. Build and train an SVM using Quantum Kernel Methods.

In this tutorial, we use `tq.op_name_dict`, `tq.functional.func_name_dict` and `tq.QuantumDevice` from TorchQuantum.

You can learn how to build a Quantum kernel function and train an SVM with the quantum kernel from this tutorial.


##Introduction to Quantum Kernel Methods.


###Kernel Methods
Kernels or kernel methods (also called Kernel functions) are sets of different types of algorithms that are being used for pattern analysis. They are used to solve a non-linear problem by a linear classifier. Kernels Methods are employed in SVM (Support Vector Machines) which are often used in classification and regression problems. The SVM uses what is called a “Kernel Trick” where the data is transformed and an optimal boundary is found for the possible outputs.


####Quantum Kernel
Quantum circuit can transfer the data to a high dimension Hilbert space which is hard to simulate on classical computer. Using kernel methods based on this Hilbert space can achieve unexpected performance.

###How to evaluate the distance in Hilbert space?
Assume S(x) is the unitary that transfer data x to the state in Hilbert space. To evaluate the inner product between S(x) and S(y), we add a Transpose Conjugation of S(y) behind S(x) and measure the probability that the state falls on $|00\cdots0\rangle$


<div align="center">
<img src="https://github.com/mit-han-lab/torchquantum/blob/master/figs/kernel.png?raw=true" alt="conv-full-layer" width="600">
</div>

##Build and train an SVM using Quantum Kernel Methods.

###Installation

In [ ]:
!pip install qiskit==0.32.1

     |████████████████████████████████| 6.1 MB 4.3 MB/s 
     |████████████████████████████████| 17.9 MB 1.3 MB/s 
     |████████████████████████████████| 237 kB 62.2 MB/s 
     |████████████████████████████████| 207 kB 60.0 MB/s 
     |████████████████████████████████| 2.1 MB 55.7 MB/s 
     |████████████████████████████████| 634 kB 32.5 MB/s 
     |████████████████████████████████| 1.6 MB 52.2 MB/s 
     |████████████████████████████████| 53 kB 2.3 MB/s 
     |████████████████████████████████| 38.2 MB 1.2 MB/s 
     |████████████████████████████████| 49 kB 6.3 MB/s 
     |████████████████████████████████| 943 kB 59.9 MB/s 
     |████████████████████████████████| 3.6 MB 53.7 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
     |████████████████████████████████| 6.4 MB 23.4 MB/s 
  Created wheel for qiskit: filename=qiskit-0.32.1-py3-none-any.whl size=11777 sha256=16a15d5cc4c9bdab55b6da049c7e58a7c58cd02e22ecc9044718a27788244262
  Stored in directory: /root/.cache/pip/whee

Download and cd to the repo.

In [ ]:
!git clone https://github.com/mit-han-lab/pytorch-quantum.git

Cloning into 'pytorch-quantum'...
remote: Enumerating objects: 10724, done.
remote: Counting objects: 100% (7516/7516), done.
remote: Compressing objects: 100% (3769/3769), done.
remote: Total 10724 (delta 3756), reused 7069 (delta 3343), pack-reused 3208
Receiving objects: 100% (10724/10724), 3.19 MiB | 21.64 MiB/s, done.
Resolving deltas: 100% (5723/5723), done.
Checking out files: 100% (50054/50054), done.


In [ ]:
%cd pytorch-quantum

/content/pytorch-quantum


Install torch-quantum.

In [ ]:
!pip install --editable .

Obtaining file:///content/pytorch-quantum
     |████████████████████████████████| 11.2 MB 7.9 MB/s 
     |████████████████████████████████| 81 kB 4.9 MB/s 
     |████████████████████████████████| 895 kB 49.7 MB/s 
     |████████████████████████████████| 65 kB 2.6 MB/s 
     |████████████████████████████████| 296 kB 59.7 MB/s 
     |████████████████████████████████| 58 kB 6.4 MB/s 
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
  Running setup.py develop for torchquantum
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


Change PYTHONPATH and install other packages.

In [ ]:
%env PYTHONPATH=.

env: PYTHONPATH=.


Run the following code to store a qiskit token. You can replace it with your own token from your IBMQ account if you like.



In [ ]:
from qiskit import IBMQ
IBMQ.save_account('0238b0afc0dc515fe7987b02706791d1719cb89b68befedc125eded0607e6e9e9f26d3eed482f66fdc45fdfceca3aab2edb9519d96b39e9c78040194b86e7858', overwrite=True)

###Import the module
`SVC` is support vector classification. We use this module to call the support vector machine algorithm.

`load_iris` is to load the famous iris dataset.

`StandardScaler` is to help scale the data by removing the mean and scaling to unit variance.

`train_test_split` is a tool to split the dataset.

`accuracy_score` can check how many samples are correctly predicted and give us the accuracy.

`func_name_dict` is a very important dict under `torchquantum.functional`. If we feed the name of the gates we want, like ‘rx’, ‘ry’, or ‘rzz’, the dict will give us a function. The function plays a central role in our quantum model. It performs the specified unitary operations on a specified quantum state on a specified wire. These three specified things are the three parameters we need to pass to it. You can see that later.


In [ ]:
import numpy as np
import torch

from sklearn.svm import SVC
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from torchquantum.functional import func_name_dict
import torchquantum as tq

###Prepare dataset
We use the front 100 samples of IRIS dataset.

Since the phase in quantum gates is 2π-periodic, it is necessary to scale the data in a range from -π to π.

And we change the label from 0 and 1 to -1 and 1.

Split the dataset on a 3-to-1 ratio.


In [ ]:
X, y = load_iris(return_X_y=True)

X = X[:100]
y = y[:100]

scaler = StandardScaler().fit(X)
X_scaled = scaler.transform(X)
y_scaled = 2 * (y - 0.5)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled)

### Build the Ansatz, consist of a unitary and its transpose conjugation.
When initializing the `KernelAnsatz`, we only need to pass a `func_list` and the KernelAnsatz will record the `func_list`. Each entry in the function is a dict, containing 'input_idx', 'func', and 'wires'.

When executing the `KernelAnsatz`, three parameters are passed from outside, `q_device`, `x`, and `y`. `q_device` stores the state vector. We reset the state vector to the $|00\cdots0⟩$. And if you didn’t forget the figure, we will act the S(x) and the transpose conjugation of S(y) to the `q_device`.

Here the gates in the `func_list` with data `x` form the unitary S(x). S(y)'s transpose conjugation is S(y)’s inverse matrix. From the perspective of inverse, we can build the S(y)'s transpose conjugation by inverting the function list with data `y`. So, how to invert a list of gates executed from head to tail? You only need to counteract the list of gates from tail to head one by one. So here, we simply reverse the sequence of function list and flip the phase from positive to negative or from negative to positive.

And in each iteration is to act the unitary gate on the quantum state. We look up the `func_name_dict` with the function name. Here the function name is the gate name, like ‘ry’, ‘rz’ and so so. The dict returns a function. We pass the three parameters to the function and the function will act the gate on the state vector(`self.q_device`), on the `wires`, with the phase, here the `params` mean phase.


In [ ]:
class KernalAnsatz(tq.QuantumModule):
    def __init__(self, func_list):
        super().__init__()
        self.func_list = func_list
    
    @tq.static_support
    def forward(self, q_device: tq.QuantumDevice, x, y):
        self.q_device = q_device
        self.q_device.reset_states(x.shape[0])
        for info in self.func_list:
            if tq.op_name_dict[info['func']].num_params > 0:
                params = x[:, info['input_idx']]
            else:
                params = None
            func_name_dict[info['func']](
                self.q_device,
                wires=info['wires'],
                params=params,
            )
        for info in reversed(self.func_list):
            if tq.op_name_dict[info['func']].num_params > 0:
                params = -y[:, info['input_idx']]
            else:
                params = None
            func_name_dict[info['func']](
                self.q_device,
                wires=info['wires'],
                params=params,
            )

### Build the whole quantum circuit

The whole model initialization is a 4-wire quantum state, the `tq.QuantumDevice` module can store the state vector and a `KernelAnsatz` we just mentioned.

When executing the whole model, as there’s a concept of batch in torchquantum’s model, we set the batch size is 1. After executing the `KernelAnsatz`, we measure the probability that the quantum state falls on the $|00\cdots0\rangle$ as the result. We get the state vector, flatten it, get the first amplitude, which is also the amplitude of the $|00\cdots0\rangle$ state, calculate the absolute value of the amplitude, and get the probability that the quantum state falls on the $|00\cdots0\rangle$ state.


In [ ]:
class Kernel(tq.QuantumModule):
    def __init__(self):
        super().__init__()
        self.n_wires = 4
        self.q_device = tq.QuantumDevice(n_wires=self.n_wires)
        self.ansatz = KernalAnsatz(
        [   {'input_idx': [0], 'func': 'ry', 'wires': [0]},
            {'input_idx': [1], 'func': 'ry', 'wires': [1]},
            {'input_idx': [2], 'func': 'ry', 'wires': [2]},
            {'input_idx': [3], 'func': 'ry', 'wires': [3]},])

    def forward(self, x, y, use_qiskit=False):
        # bsz=1
        x = x.reshape(1, -1)
        y = y.reshape(1, -1)
        self.ansatz(self.q_device, x, y)
        result = torch.abs(self.q_device.states.view(-1)[0])
        return result

###Train the svm model from sklearn based on our quantum kernel.

Define a kernel matrix function.

Pass the kernel matrix function to SVC, call `.fit(X_train, y_train)` and the SVC object can start training.

Predict and see the accuracy. The accuracy looks pretty well.


In [ ]:
kernel_function = Kernel()
def kernel_matrix(A, B):
    return np.array([[kernel_function(a, b) for b in B] for a in A])

svm = SVC(kernel=kernel_matrix).fit(X_train, y_train)
predictions = svm.predict(X_test)
print(accuracy_score(predictions, y_test))

1.0
